# Calibration based on One Touch Knock Out Daily Cliquet Options

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

Let's initialize the values found in StockAnalysis module: scale factors for otko option prices and spx values.

In [2]:
scale_factors = {
    'TSLA': 10.35,
    'META': 6.23,
    'AAPL': 7.84
}

spx_data = pd.read_csv('data/SPX_data.csv')
print(spx_data)

           Date         Open         High          Low        Close   
0    2016-01-20  1876.180054  1876.180054  1812.290039  1859.329956  \
1    2016-01-21  1861.459961  1889.849976  1848.979980  1868.989990   
2    2016-01-22  1877.400024  1908.849976  1877.400024  1906.900024   
3    2016-01-25  1906.280029  1906.280029  1875.969971  1877.079956   
4    2016-01-26  1878.790039  1906.729980  1878.790039  1903.630005   
..          ...          ...          ...          ...          ...   
500  2018-01-12  2770.179932  2787.850098  2769.639893  2786.239990   
501  2018-01-16  2798.959961  2807.540039  2768.639893  2776.419922   
502  2018-01-17  2784.989990  2807.040039  2778.379883  2802.560059   
503  2018-01-18  2802.399902  2805.830078  2792.560059  2798.030029   
504  2018-01-19  2802.600098  2810.330078  2798.080078  2810.300049   

         Volume  
0    6416070000  
1    5078810000  
2    4901760000  
3    4401380000  
4    4357940000  
..          ...  
500  3587220000  
501

Let's retrieve the prices for One Touch Knock Out options on the SPX.

In [3]:
otko_spx = pd.read_csv('data\otko_prices_spx.csv')
print(f'Otko Prices: \n')
print(otko_spx.head())
SPX_S0 = spx_data.loc[spx_data['Date'] == '2016-01-20', 'Close'].values[0]
print(f'\n SPX Closing Value on 20/01/2016:', round(SPX_S0, 2))

Otko Prices: 

  Starting date Stock Name Expiry  K1  K2      BID      ASK
0    20/01/2016        SPX     6M  75   0  0.0850%  0.1000%
1    20/01/2016        SPX     6M  70   0  0.0400%  0.0500%
2    20/01/2016        SPX     6M  85  75  0.1700%  0.2000%
3    20/01/2016        SPX     6M  80  70  0.1200%  0.1400%
4    20/01/2016        SPX     6M  90  80  0.2400%  0.2700%

 SPX Closing Value on 20/01/2016: 1859.33


The BID(%) and ASK(%) prices are in percentages. This means that it will be $BID*S(0)$ and $ASK*S(0)$ where S(0) is the SPX market value at the starting date.

## Refactoring dataset

In [4]:
# Remove the percentage symbol and convert the columns to float
otko_spx['BID'] = otko_spx['BID'].str.rstrip('%').astype(float)
otko_spx['ASK'] = otko_spx['ASK'].str.rstrip('%').astype(float)

# Refactor to 1YEAR expiry
otko_spx['Expiry'] = '1Y'
otko_spx['BID'] = 2 * otko_spx['BID']
otko_spx['ASK'] = 2 * otko_spx['ASK']

# Rename the columns
otko_spx.rename(columns={'BID': 'BID(%)', 'ASK': 'ASK(%)'}, inplace=True)

otko_spx['BID'] = otko_spx['BID(%)'] * SPX_S0 / 100
otko_spx['ASK'] = otko_spx['ASK(%)'] * SPX_S0 / 100

otko_spx['Midpoint'] = otko_spx['BID'] + (abs(otko_spx['BID'] - otko_spx['ASK']) / 2)

otko_spx.to_csv('data/otko_spx_1year.csv')
print(otko_spx)

  Starting date Stock Name Expiry  K1  K2  BID(%)  ASK(%)       BID   
0    20/01/2016        SPX     1Y  75   0    0.17    0.20  3.160861  \
1    20/01/2016        SPX     1Y  70   0    0.08    0.10  1.487464   
2    20/01/2016        SPX     1Y  85  75    0.34    0.40  6.321722   
3    20/01/2016        SPX     1Y  80  70    0.24    0.28  4.462392   
4    20/01/2016        SPX     1Y  90  80    0.48    0.54  8.924784   

         ASK  Midpoint  
0   3.718660  3.439760  
1   1.859330  1.673397  
2   7.437320  6.879521  
3   5.206124  4.834258  
4  10.040382  9.482583  
